In [4]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.functions import KernelArguments

from dotenv import load_dotenv
import os


In [5]:
kernel = Kernel()

load_dotenv()
kernel.add_service(
    AzureChatCompletion(service_id="multi_functions", 
                        deployment_name=os.getenv("DEPLOYMENT_NAME"), 
                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
                        api_key=os.getenv("AZURE_OPENAI_API_KEY"))
)

##### Travel Function

In [6]:
from semantic_kernel.functions import kernel_function

class TravelGuide:
    """Gives Weather, time and currency info """
    @kernel_function
    def get_weather(self, city: str) -> str:
        weather_data = {
            "London": "15 degrees and rainy",
            "Dubai": "35 degrees and sunny",
            "New York": "10 degrees and snowy"
        }
        return weather_data.get(city, "22 degrees and partly cloud")
    
    @kernel_function
    def get_time(self, city: str) -> str:
        return f"Current time in {city} is 10 AM. "
    
    @kernel_function
    def get_currency(self, country: str) -> str:
        currency = {
            "UK": "Euro", 
            "USA": "USD", 
            "Dubai": "Dirams"
        }
        return f"Currency in {country} is {currency.get(country, 'Unknown')}"

In [ ]:
import random
class GenerateNumberPlugin:
    @kernel_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

In [8]:
class MathPlugin:
    @kernel_function
    def add(self, num1: int, num2: int) -> int:
        return num1 + num2
    
    @kernel_function
    def subtract(self, num1: int, num2: int) -> int:
        return num1-num2

In [ ]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

async def main():
    kernel = Kernel()

    # Configure AI Service
    kernel.add_service(AzureChatCompletion(
        service_id="agent_brain",
        deployment_name="gpt-4.1",
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    ))

    # Register ALL 3 Plugins
    kernel.add_plugin(TravelGuide(), plugin_name="Travel")
    kernel.add_plugin(GenerateNumberPlugin(), plugin_name="Random")
    kernel.add_plugin(MathPlugin(), plugin_name="Math")

    # Set Behavior to AUTO (The "Decision Maker")
    settings = kernel.get_prompt_execution_settings_from_service_id("agent_brain")
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()


    query1 = "What's the weather like in London?"
    
    query2 = "What is 1542 plus 893?"
    
    query3 = "Generate a random number between 3 and 100, then add 50 to it."

    for query in [query1, query2, query3]:
        print(f"\nUser: {query}")
        response = await kernel.invoke_prompt(prompt=query, settings=settings)
        print(f"Agent: {response}")


await main()


User: What's the weather like in London?
Agent: The weather in London is currently 15 degrees Celsius and rainy. If you’re heading out, make sure to bring an umbrella!

User: What is 1542 plus 893?
Agent: 1542 plus 893 equals 2435.

User: Generate a random number between 3 and 100, then add 50 to it.
Agent: The randomly generated number between 3 and 100 is 98. After adding 50, the result is 148.


: 